## Install BRAILS++
Before running the following cells, install the latest version of BRAILS++.

In [ ]:
!pip install brails

# Import BRAILS Importer

In [ ]:
from brails.utils import Importer

# Define Location Specific Parameters

In [ ]:
LOCATION_NAME = 'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'FortMyersInventory_HU.geojson'
NO_POSSIBLE_WORLDS = 0

# Create and Importer object to Pull In Required BRAILS Modules

In [ ]:
importer = Importer()

# Define the Region Object for the Region of Interest

In [ ]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

# Get Raw NSI Data for the Defined Region

In [ ]:
nsi_class = importer.get_class('NSI_Parser')()
nsi_inventory = nsi_class.get_raw_data(region_boundary_object)

# Get FEMA USA Footprints Data for the Defined Region

In [ ]:
footprint_scraper = importer.get_class('USA_FootprintScraper')({'length': 'ft'})
footprint_inventory = footprint_scraper.get_footprints(region_boundary_object)

# Create a Baseline Inventory by Merging NSI Raw Data and USA Structures Footprints

In [ ]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    footprint_inventory, "ft", get_extended_features=True, add_features=[])

# Fill Gaps in the Baseline Inventory by Imputing Missing Values

In [ ]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

# Dowload Aerial Imagery for the Identified Assets

In [ ]:
google_satellite_class = importer.get_class('GoogleSatellite')()
images_aerial = google_satellite_class.get_images(
    imputed_inventory,
    'tmp/satellite/'
)

# Predict Roof Type Information from Aerial Imagery

In [ ]:
roof_shape_classifier = importer.get_class('RoofShapeClassifier')()
predictions = roof_shape_classifier.predict(images_aerial)

# Getting Design Wind Speed, Flood Zone and Roof Type Information ,..., in Created Inventory

In [ ]:
for key, val in imputed_inventory.inventory.items():
    val.add_features({'DesignWindSpeed': 130,
                      'FloodZone': 'AE',
                      'RoofShape': predictions[key],
                      'AvgJanTemp':'Above',
                      'NumberOfUnits':1,
                      'LandCover': 'Open'})

# Run Rulesets to Infer R2D-Required Data for Hazus Damage and Loss model

In [ ]:
imputed_inventory.change_feature_names({'fparea': 'PlanArea',
                                        'erabuilt': 'YearBuilt',
                                        'constype': 'BuildingType',
                                        'numstories': 'NumberOfStories',
                                        'occupancy': 'OccupancyClass'})
hurricaneInferer = importer.get_class("HazusInfererWind")
inferer = hurricaneInferer(
    input_inventory=imputed_inventory, clean_features=True)
hazus_inferred_inventory = inferer.infer()

# Validate the inventory and make correction if needed

In [ ]:
invalid_id, error_record = inferer.validate(hazus_inferred_inventory)

In [ ]:
inventory_corrrected = inferer.correct(hazus_inferred_inventory, invalid_id=invalid_id, weights={'StructureType':2})

# Write the Created Inventory in a GeoJSON File

In [ ]:
_ = inventory_corrrected.write_to_geojson(
    output_file=INVENTORY_OUTPUT)